In [118]:
################################################################################
# CSE 253: Programming Assignment 1
# Code snippet by Jenny Hamer
# Winter 2019
################################################################################
# We've provided you with the dataset in CAFE.tar.gz. To uncompress, use:
# tar -xzvf CAFE.tar.gz
################################################################################
# To install PIL, refer to the instructions for your system:
# https://pillow.readthedocs.io/en/5.2.x/installation.html
################################################################################
# If you don't have NumPy installed, please use the instructions here:
# https://scipy.org/install.html
################################################################################

from os import listdir
from PIL import Image
import numpy as np
from scipy import linalg as la

# The relative path to your CAFE-Gamma dataset
data_dir = "./CAFE/"

# Dictionary of semantic "label" to emotions
emotion_dict = {"h": "happy", "ht": "happy with teeth", "m": "maudlin",
	"s": "surprise", "f": "fear", "a": "anger", "d": "disgust", "n": "neutral"}


def load_data(data_dir="./CAFE/"):
    """ Load all PGM images stored in your data directory into a list of NumPy
    arrays with a list of corresponding labels.

    Args:
        data_dir: The relative filepath to the CAFE dataset.
    Returns:
        images: A list containing every image in CAFE as an array.
        labels: A list of the corresponding labels (filenames) for each image.
    """
    # Get the list of image file names
    all_files = listdir(data_dir)

    # Store the images as arrays and their labels in two lists
    images = []
    labels = []

    for file in all_files:
    # Load in the files as PIL images and convert to NumPy arrays
        if file.find('_ht') == -1 and file.find('_n') == -1:
            img = Image.open(data_dir + file)
            images.append(np.array(img))
            labels.append(file)

    print("Total number of images:", len(images), "and labels:", len(labels))

    return images, labels




In [4]:
data.shape

NameError: name 'data' is not defined

In [105]:
###### TODO  DEVIDE Eigen VEctore to EigenCVALUE

def PCA(data, dims_rescaled_data=2):
    """
    returns: data transformed in 2 dims/columns + regenerated original data
    pass in: data as 2D NumPy array
    """
    # mean center the data
    mean = data.mean(axis=1)
    std = data.std(axis=1)
    data = data-mean
    data = data_T.transpose()

    # calculate the covariance matrix
    print(data_T.shape)
    print(data.shape)
    R = np.dot(data_T,data)
    evals, evecs = LA.eigh(R)
    # sort eigenvalue in decreasing order
    idx = NP.argsort(evals)[::-1]
    vect = NP.dot(data,evecs)
    evecs = vect[:,idx]
    # print(evecs.shape)
    evals = evals[idx]
    print(evals)
    evecs = evecs/np.sqrt(evals) 
    evecs = evecs[:,:dims_rescaled_data]
    ##vect = vect-vect.mean(axis = 0)
    #vect = vect[:,:dims_rescaled_data]
    result = NP.dot(data_T,evecs[:,0])
    result = result/np.linalg.norm(evecs[:,0])
    #result = result - result.mean(axis =0) 
    return result,evecs,evals,mean,std

def test_PCA(data, dims_rescaled_data=2):
    '''
    test by attempting to recover original data array from
    the eigenvectors of its covariance matrix & comparing that
    'recovered' array with the original data
    '''
    _ , _ , eigenvectors = PCA(data, dim_rescaled_data=2)
    data_recovered = NP.dot(eigenvectors, m).T
    data_recovered += data_recovered.mean(axis=0)
    assert NP.allclose(data, data_recovered)


def plot_pca(data):
    from matplotlib import pyplot as MPL
    clr1 =  '#2026B2'
    fig = MPL.figure()
    ax1 = fig.add_subplot(111)
    data_resc, data_orig = PCA(data)
    ax1.plot(data_resc[:, 0], data_resc[:, 1], '.', mfc=clr1, mec=clr1)
    MPL.show()


In [260]:
dims_rescaled_data=47
A = im_re.T
mean = A.mean(axis=1)
std = A.std(axis=1)
A = (A.T-mean).T
A_T = A.T
sm = np.matmul(A_T,A)
evals, ev = la.eigh(sm)
idx = np.argsort(evals)[::-1]
evals = evals[idx]
ev = ev[idx,:]
Av = np.matmul(A,ev)
evct = Av[:,:dims_rescaled_data]
result = np.matmul(A_T,evct)

In [261]:
np.inner(A_T,v_p).std()

0.1443375672974064

In [265]:
sss=np.matmul(A.T,np.matmul(A,ev[0,:])/(np.linalg.norm(np.matmul(A,ev[0,:]))))

In [266]:
sss.std()

2112.0045336300404

In [267]:
evals[0]

582593048.5364782

In [257]:
np.sqrt(evals[0])


24136.964360426067

In [186]:
v_p = Av[:,0]/(np.linalg.norm(Av[:,0])*np.sqrt(abs(evals[0])))

In [158]:
A.mean(axis=1).shape

(91200,)

In [97]:
import numpy as np
images, labels = load_data(data_dir="./CAFE/")
im = np.array(images[:48], 'float64')
im_re = np.reshape(im, [len(im), -1])
pca_result,evecs,evals,mean,vari = PCA(im_re.T, dims_rescaled_data=47)



Total number of images: 60 and labels: 60
(48, 91200)
(91200, 48)
[8.23494948e+10 7.87809813e+05 6.33172878e+05 3.26764563e+05
 2.91863814e+05 2.56301671e+05 2.03300669e+05 1.91288501e+05
 1.48104871e+05 1.14227680e+05 1.02537741e+05 8.65584207e+04
 7.84449374e+04 6.75922432e+04 6.16872154e+04 5.91331019e+04
 5.58296048e+04 4.80302070e+04 4.77297630e+04 4.47134600e+04
 4.29948917e+04 3.86110331e+04 3.53881062e+04 3.31248011e+04
 3.21962328e+04 2.83607552e+04 2.80496441e+04 2.64889040e+04
 2.44715825e+04 2.35041584e+04 2.18761434e+04 2.11986624e+04
 2.07822491e+04 1.96044163e+04 1.84735261e+04 1.80207862e+04
 1.67368282e+04 1.63485663e+04 1.58332428e+04 1.48968599e+04
 1.39348043e+04 1.28485074e+04 1.19914975e+04 1.11462246e+04
 1.04630522e+04 1.02449745e+04 8.39249784e+03 7.54069013e+03]


In [59]:
evecs.shape

(91200, 47)

In [60]:
evecs_pic = np.reshape(evecs.T[0:6,:],[6,380,-1])
def display_face(img):
    """ Display the input image and optionally save as a PNG.

    Args:
    img: The NumPy array or image to display

    Returns: None
    """
    # Convert img to PIL Image object (if it's an ndarray)
    if type(img) == np.ndarray:
        print("Converting from array to PIL Image")
        im = (img - img.min())*(255/(img.max()-img.min()))
        # normalize the img into 0-255
        img = Image.fromarray(im)
    # Display the image
    img.show()
display_face(evecs_pic[0,:,:])
display_face(evecs_pic[1,:,:])
display_face(evecs_pic[2,:,:])
display_face(evecs_pic[3,:,:])
display_face(evecs_pic[4,:,:])
display_face(evecs_pic[5,:,:])

Converting from array to PIL Image
Converting from array to PIL Image
Converting from array to PIL Image
Converting from array to PIL Image
Converting from array to PIL Image
Converting from array to PIL Image


In [61]:
import math
def get_sad(images, labels):
    image_sad = []
    for i in range(len(images)):
        if labels[i].find('_m') != -1:
            image_sad.append(images[i])
    return image_sad
sad_vector = get_sad(images, labels)
sad =np.array(sad_vector,'float64')
def get_happy(images, labels):
    image_happy = []
    for i in range(len(images)):
        if labels[i].find('_h') != -1:
            image_happy.append(images[i])
    return image_happy
happy_vector = get_happy(images, labels)
happy=np.array(happy_vector,'float64')
from random import shuffle
def shuffle_n_generate_data(train, target):
    length = len(train)
    train = np.array(train,'float64')
    target = np.array(target)
    ind_list = [i for i in range(length)]
    shuffle(ind_list)
    train_new  = train[ind_list,:,:]
    target_new = target[ind_list]
    return train_new, target_new

## 2.b.(1)

In [50]:
features_pca.std()

22757.63780683967

In [70]:
def sigmoid(x): 
    return 1. / (1. + np.exp(-x))
def loss(t, y):
    return -(t * np.log(y) +(1 - t) * np.log(1 - y)).mean()
def accuracy(t,p):
    return 1-1.0*sum(abs(t-(p>0.5)))/len(t)
def logistic_regression(features,target,handout,handout_label, epoch, learning_rate):
    ce_loss = []
    best_weight = np.zeros(features.shape[1])  
    weights = np.zeros(features.shape[1])
    hand_out_mark = float('inf')
    hand_out_loss = []
    m = features.shape[0]
    for step in range(epoch):
        scores = np.dot(features, weights)
        predictions = sigmoid(scores)
        print(predictions)
        # record Cross-Entropy Error
        ce_loss.append(loss(target,sigmoid(np.dot(features,weights))))
        # Update weights with gradient 
        ##temp = loss(handout_label,sigmoid(np.dot(handout,weights)))
        #print(sigmoid(np.dot(handout,weights)))
        ##if temp < hand_out_mark:
        ##    hand_out_mark = temp
        ##    best_weight = weights
        ##print(sum(predictions>0.5))
        gradient = 1/m*np.dot(features.T, predictions-target)
        #print(gradient)
        weights += learning_rate * gradient 
        ##hand_out_loss.append(temp)
    return weights, best_weight,ce_loss,hand_out_loss


In [71]:
image_logistic = np.concatenate((sad,happy),axis=0)
label_logistic = np.concatenate(([0]*10,[1]*10),axis=0)
[image_logistic_s,label_logistic_s] = shuffle_n_generate_data(image_logistic, label_logistic)

devision = int(0.8*len(image_logistic_s))
devision_2 = int(0.9*len(image_logistic_s))
train_logistic  = image_logistic_s[:devision,:,:]
train_label_logistic = label_logistic_s[:devision]
handout_logistic = image_logistic_s[devision:devision_2,:,:]
handout_label_logistic = label_logistic_s[devision:devision_2]
test_logistic  = image_logistic_s[devision_2:,:,:]
test_label_logistic = label_logistic_s [devision_2:]

[features_pca,evect,evals,train_mean,train_std] = PCA(np.reshape(train_logistic,[len(train_logistic),-1]), dims_rescaled_data=15)
handout_logistic_b = np.reshape(handout_logistic,[len(handout_logistic),-1])/train_std-train_mean
handout_logistic_pca = np.matmul((handout_logistic_b),evect)
[weight,best_weight,ce_loss,handout_loss] = logistic_regression(features_pca,train_label_logistic,handout_logistic_pca,handout_label_logistic,epoch=10, learning_rate=0.1)



(16, 91200)
(91200, 16)
[2.76398290e+10 2.84240879e+05 2.74237415e+05 1.47775000e+05
 1.21057005e+05 1.07198668e+05 9.66391534e+04 7.44017702e+04
 6.20724373e+04 4.61201388e+04 4.06192006e+04 3.36198852e+04
 2.92923649e+04 2.42933753e+04 1.95410348e+04 1.59610960e+04]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in multiply
  after removing the cwd from sys.path.


In [69]:
features_pca.shape

(16, 15)

In [103]:
handout_logistic_a = np.reshape(handout_logistic,[len(handout_logistic),-1])

In [77]:
handout_logistic_pca


array([[45053581.10101586, -3603112.80100734,  -735854.05450527,
        -3979448.60718387, -3687271.41808088, -4494244.86819231,
        -1241669.56956167, -3934353.6138574 , -3672394.78184577,
        -3229022.74824177, -3547880.25504816, -3462365.49311423,
        -3623258.47583446, -2680289.20982047, -3162415.20472226],
       [45150856.73845325, -3617695.55988003,  -732429.24640976,
        -3989378.7325359 , -3698747.55293883, -4501697.91999709,
        -1240962.9921502 , -3942604.35813272, -3684597.70532424,
        -3234966.09637943, -3554062.6670973 , -3469440.91868556,
        -3630569.88038068, -2684939.51562853, -3168763.59291294]])

In [ ]:
image_logistic = np.concatenate((sad,happy),axis=0)
label_logistic = np.concatenate(([0]*10,[1]*10),axis=0)
loss_train = []
loss_val=[]
test_accuracy = []
for i in range (10):
    [image_logistic_s,label_logistic_s] = shuffle_n_generate_data(image_logistic, label_logistic)
    
    devision = int(0.8*len(image_logistic_s))
    devision_2 = int(0.9*len(image_logistic_s))
    train_logistic  = image_logistic_s[:devision,:,:]
    train_label_logistic = label_logistic_s[:devision]
    handout_logistic = image_logistic_s[devision:devision_2,:,:]
    handout_label_logistic = label_logistic_s[devision:devision_2]
    test_logistic  = image_logistic_s[devision_2:,:,:]
    test_label_logistic = label_logistic_s [devision_2:]
    
    [features_pca,evect,train_mean,train_std] = \
    PCA(np.reshape(train_logistic,[len(train_logistic),-1]), dims_rescaled_data=15)
    
    handout_logistic = np.reshape(handout_logistic,[len(handout_logistic),-1])/train_std
    handout_logistic_pca = np.dot(handout_logistic,evect.T)
    
    [weight,best_weight,ce_loss,handout_loss] = \
    logistic_regression(features_pca,labels_1,handout_logistic_pca,\
                        handout_label_logistic, epoch=10, learning_rate=0.1)
    
    loss_train.append(ce_loss)
    loss_val.append(handout_loss)
    
    test_logistic = np.reshape(test_logistic,[len(test_logistic),-1])/train_std
    test_logistic_pca = np.dot(test_logistic,evect.T)
    
    test_result = sigmoid(np.dot(test_logistic_pca,weight))
    test_accuracy.append(accuracy(test_label_logistic,test_result))
    #print(test_result)
loss_avg = loss_overall.mean(axis=0)
loss_val_avg = loss_val.mean(axis=0)  


In [840]:
test_accuracy

[0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0]

In [615]:
## TODO here for ten times, we need to random pick the training data and validation data.
features = np.concatenate((sad[:8],happy[:8]),axis=0)
labels_1 = np.concatenate(([0]*8,[1]*8),axis=0)

In [130]:
holdout_feature = np.concatenate((sad[7:8],happy[7:8]),axis=0)
holdout_label = np.concatenate(([0],[1]),axis=0)

In [131]:
test_feature = np.concatenate((sad[8:9],happy[8:9]),axis=0)
test_label = np.concatenate(([0],[1]),axis=0)

In [160]:
features_pca, evect = PCA(features, dims_rescaled_data=15)

(16,)
(16, 16)
(16, 91200)
(91200, 15)
(16, 15)


In [213]:
loss_overall=[]
for i in range(10):
    [weight,ce_loss] = logistic_regression(features_pca, labels_1, 10, 0.1, add_intercept = False)
    loss_overall.append(ce_loss)
    ## TODO need to know how many graph need to print.

In [198]:
holdout_pca = np.dot(holdout_feature,evect)
holdout_pca = holdout_pca - holdout_pca.mean()
holdout_pca = holdout_pca / holdout_pca.std()

In [164]:
holdout_pca

array([[-3.59456151,  0.82955879,  1.50045876,  0.37629237,  0.27772601,
        -0.08406059, -0.45896371,  0.04358075,  0.11146505, -0.29916364,
        -0.98522272,  0.99948702, -0.06402932, -0.05704641,  0.29104213],
       [-1.97691541,  2.04132495, -1.1899904 ,  0.40191635,  0.53067138,
        -0.13887111,  0.37930887,  0.84190681,  0.20134868,  0.08671695,
         0.31215654,  0.27197625, -0.68661055,  0.15992016, -0.12142244]])

In [165]:
holdout_result = sigmoid(np.dot(holdout_pca,weight)) 

In [166]:
holdout_result

array([0.1421452 , 0.67520279])

In [324]:
from random import shuffle
def shuffle_n_generate_data(train, target):
    length = len(train)
    train = np.array(train,'float64')
    target = np.array(target)
    ind_list = [i for i in range(length)]
    shuffle(ind_list)
    train_new  = train[ind_list,:,:]
    target_new = target[ind_list,]
    return train_new, target_new

In [325]:
def get_data_for_softmax_re(images, labels):
    train = []
    target = []
    for i in range(len(images)):
        if labels[i].find('_ht') == -1 and labels[i].find('_n') == -1 :
            train.append(images[i])
            if labels[i].find('_h') !=-1:
                target.append(0)
            elif labels[i].find('_m') !=-1:
                target.append(1)
            elif labels[i].find('_s') !=-1:
                target.append(2)
            elif labels[i].find('_f') !=-1:
                target.append(3)
            elif labels[i].find('_a') !=-1:
                target.append(4)
            elif labels[i].find('_d') !=-1:
                target.append(5)
    return train, target

In [326]:
[image_softmax, label_softmax] = get_data_for_softmax_re(images, labels)

In [327]:
image_softmax,label_softmax= shuffle_n_generate_data(image_softmax, label_softmax)

(60,)


In [328]:
devision = int(0.8*len(image_softmax))
devision_2 = int(0.9*len(image_softmax))
train_softmax  = image_softmax[:devision,:,:]
train_label_softmax  = label_softmax[:devision]
handout_softmax  = image_softmax[devision:devision_2,:,:]
handout_label_softmax  = label_softmax[devision:devision_2]
test_softmax  = image_softmax[devision_2:,:,:]
test_label_softmax  = label_softmax[devision_2:]

In [329]:
train_softmax.shape

(48, 380, 240)

In [594]:
def oneHot(Y):
    result = []
    for i in range(len(Y)):
        onehot = [0]*(int(Y.max())+1)
        onehot[Y[i]] = 1
        result.append(onehot)
    return result

def cross_entropy(x, y):
    """ Computes cross entropy between two distributions.
    Input: x: iterabale of N non-negative values
           y: iterabale of N non-negative values
    Returns: scalar
    """

    if np.any(x < 0) or np.any(y < 0):
        raise ValueError('Negative values exist.')

    # Force to proper probability mass function.
    x = np.array(x, dtype=np.float)
    y = np.array(y, dtype=np.float)
    x /= np.sum(x)
    y /= np.sum(y)

    # Ignore zero 'y' elements.
    mask = y > 0
    x = x[mask]
    y = y[mask]    
    ce = -np.sum(x * np.log(y)) 
    return ce
def softmax(x):
    s = np.max(x, axis=1)
    s = s[:, np.newaxis] # necessary step to do broadcasting
    e_x = np.exp(x - s)
    div = np.sum(e_x, axis=1)
    div = div[:, np.newaxis] # dito
    return e_x / div
def sofmat_regression(x,y, epoch,learningRate):
    losses = []
    weight = np.zeros([x.shape[1],len(np.unique(y))])
    m = x.shape[0] #First we get the number of training examples
    y_mat = oneHot(y) #Next we convert the integer class coding into a one-hot representation
    for i in range(0,epoch):
        scores = np.dot(x,weight) #Then we compute raw class scores given our input and current weights
        prob = softmax(scores) #Next we perform a softmax on these scores to get their probabilities
        loss = -1/m/6*cross_entropy(np.array(y_mat),np.log(prob)) #We then find the loss of the probabilities
        grad = -1/m/6*np.dot(x.T,(y_mat-prob))  #And compute the gradient for that loss
        losses.append(loss)
        print(loss)
        weight = weight - (learningRate * grad)
    return weight, loss

In [595]:
features_softmax_pca, evect_softmax = PCA(np.reshape(train_softmax,[len(train_softmax),-1]), dims_rescaled_data=47)

(48,)
(48, 48)
(48, 91200)
(91200, 47)
(48, 47)


In [604]:
[weight, loss_sofmax] = sofmat_regression(features_softmax_pca,train_label_softmax,epoch=50,learningRate=0.000005)

ValueError: Negative values exist.

In [603]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x.T - np.max(x).T)
    return np.transpose(e_x / e_x.sum())
def sofmat_regression_sdg(x,y, epoch,learningRate):
    losses = []
    weight = np.zeros([x.shape[1],len(np.unique(y))])
    m = x.shape[0] #First we get the number of training examples
    y_mat = oneHot(y) #Next we convert the integer class coding into a one-hot representation
    ind_list = [i for i in range(m)]
    for i in range(0,epoch):
        shuffle(ind_list)
        for j in range(m):
            scores = np.dot(x[ind_list[j]],weight) #Then we compute raw class scores given our input and current weights
            prob = softmax(scores) #Next we perform a softmax on these scores to get their probabilities
            loss = -1*np.sum(y_mat[ind_list[j]]*np.log(prob)) #We then find the loss of the probabilities
            #print(y_mat[ind_list[j]])
            #print(prob.sum())
            grad = -1*np.outer(x[ind_list[j]],(y_mat[ind_list[j]] - prob))#And compute the gradient for that loss
            #print(prob)
            losses.append(loss)
            print(loss)
            weight = weight + (learningRate * grad)
    return weight, loss